In [1]:
# import zipfile
# from google.colab import drive 
# drive.mount('/content/gdrive') 

In [2]:
!pip install flask-ngrok
# Additional Dependencies
!pip install barbar torchsummary
from myImports import *
from ConvAutoencoder_v2 import ConvAutoencoder_v2
from utils import Utils
from UploadFiles import UploadFiles
util = Utils()
# Find if any accelerator is presented, if yes switch device to use CUDA or else use CPU
# device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# print(device)
# creating an instance of a flask web application
app = Flask(__name__, static_folder = 'static', template_folder='templates')
run_with_ngrok(app)
app.secret_key = "secret key"
# Get current path
path = os.getcwd()
# file Upload
UPLOAD_FOLDER = os.path.join(path, 'uploads')
# Make directory if "uploads" folder not exists
if not os.path.isdir(UPLOAD_FOLDER):
    os.mkdir(UPLOAD_FOLDER)
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
QUERYIMAGEPATH = app.config['UPLOAD_FOLDER'] + '\\QueryImage\\'

# Allowed extension you can set your own
ALLOWED_EXTENSIONS = set(['csv', 'png', 'jpg', 'jpeg', 'zip'])

transformations = util.getTransformations()


You should consider upgrading via the 'd:\programs\python\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'd:\programs\python\python.exe -m pip install --upgrade pip' command.


cpu


In [3]:
#Define the route "how to access a specific page"
@app.route('/', methods=['GET'])
def dashboard_page():
    return render_template('dashboard.html')


@app.route('/trainModelPage', methods=['GET','POST'])
def train_model_page():
    

    if request.method == "POST":
        # if 'action' in request.form:
        #     print('request.files.getlist',request.files.getlist)
        # if action == 'upload':
        action = request.form['action']
        print(action)
        if action == 'upload':
            if 'file' not in request.files:
                flash('No file parts')
                #print("sdsd")
                return redirect(request.url)
            filesize =   request.cookies.get('filesize')  
            file =  request.files['file']
            print(f"Filesize: {filesize}")
            print(file)
            
            #print("ssss", app.config['UPLOAD_FOLDER'])
            train_set = UploadFiles(app.config['UPLOAD_FOLDER'], request.files)
            #print("aAS")
            train_set_path = train_set.upload()
            #print(train_set_path)
            os.path.exists(os.path.abspath(train_set_path))
            #print('ss', train_set_path)

            session['train_set_path'] = train_set_path

            images_to_display = []
            counter = 0
            for dirname, _, filenames in os.walk(train_set_path):
                for filename in filenames:
                    if counter == 8:
                        break
                    temp_list = []
                    temp_list.append(dirname)
                    temp_list.append(filename)
                    # imagePath = os.path.join(dirname, filename)
                    images_to_display.append(temp_list)
                    counter = counter + 1

            nbrFiles = 0
            for dirname, _, filenames in os.walk(train_set_path):
                for filename in filenames:
                    nbrFiles = nbrFiles + 1
            print('nbr ',nbrFiles)
            #print(images_to_display)

            print("File uploaded")
            
            # json_images_to_display = json.dumps(images_to_display)
            # print(json_images_to_display)
            #res = make_response(jsonify(images_to_display=json_images_to_display), 200)
            return jsonify({'data': render_template('list.html', images_to_display=images_to_display, nbrFiles=nbrFiles), 'nbrFiles': nbrFiles})

        elif action == 'train':
            optimizer = request.form['optimizer']
            learnin_rate = request.form['lr']
            epochs = request.form['epochs']
            batch_size = request.form['batchsize']
            modelName = request.form['modelName']
            print(modelName)

            model_name_info_ = modelName +'_'+  optimizer +'_'+ learnin_rate +'_'+ epochs  +'_'+ batch_size
            
            
            # #TODO add function called train with parameters
            #Check if first if there any information in the session
            if "train_set_path" in session:
                train_set_path = session['train_set_path']
                #print('sldflsdjflk')
            else:
                #print('aaaaa')
                render_template('train_model.html')


                
            #TODO add function to preparing trainset intermediate DataFrame
            # preparing intermediate DataFrame
            datasetPath = Path(train_set_path)
            df = pd.DataFrame()
            myfilename = [] 
            for dirname, _, filenames in os.walk(datasetPath):
                for filename in filenames:
                    #print(os.path.join(dirname, filename))
                    myfilename.append(os.path.join(dirname, filename))
            df['image'] = myfilename
            df.head()
            #print(df)

            EPOCHS = int(epochs)
            NUM_BATCHES = int(batch_size)
            RETRAIN = False

            # EPOCHS,NUM_BATCHES,df, 
            train_set, validate_set = util.prepare_data(DF=df)

            dataloaders = {'train': DataLoader(train_set, batch_size=NUM_BATCHES, shuffle=True, num_workers=1) ,
                            'val':DataLoader(validate_set, batch_size=NUM_BATCHES, num_workers=1)
                            }

            #To show images
            # images = next(iter(DataLoader(train_set, batch_size=NUM_BATCHES, shuffle=True, num_workers=1)))
            # helper.imshow(images[31], normalize=False)

            dataset_sizes = {'train': len(train_set),'val':len(validate_set)}

            model = ConvAutoencoder_v2().to(device)

            criterion = nn.MSELoss()
            # Observe that all parameters are being optimized
            optimizer = torch.optim.Adam(model.parameters(), lr=float(learnin_rate)) if (optimizer=="Adam") else torch.optim.SGD(model.parameters(), lr=float(learnin_rate)) 
            # Decay LR by a factor of 0.1 every 7 epochs
            exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)#this was commented

            #Training Fucntions
            model, optimizer, loss = util.train_model(
                            model=model,
                            dataloaders = dataloaders,
                            dataset_sizes = dataset_sizes,
                            criterion=criterion, 
                            optimizer=optimizer, 
                            scheduler=exp_lr_scheduler,
                            num_epochs=EPOCHS)

            
            #Save the trained model 
            torch.save({ 'epoch': EPOCHS, 'model_state_dict': model.state_dict(), 'optimizer_state_dict': optimizer.state_dict(), 'loss': loss, }, app.config['UPLOAD_FOLDER']+'/Models/'+model_name_info_+'.pt')
            res = make_response(jsonify({"message": "Model Trained"}), 200)
            return res

        else: print('welcome to bouzidia')

        #return res
    return render_template('train_model.html') #,images_to_display=images_to_display
    #return redirect(url_for('train_and_save'))


@app.route('/searchPage', methods=['GET'])
def search_page():
    models_folder_path = app.config['UPLOAD_FOLDER']+'\\Models\\'
    list_of_models = []
    for path,dirs,files in os.walk(models_folder_path):
        for filename in files:
            temp_list = []
            temp_list.append(filename)
            temp_list.append(path)
            list_of_models.append(temp_list)
            print(os.path.join(path,filename))

    return render_template('retreive_images_old.html', list_of_models = list_of_models)


@app.route('/upload/<path>/<filename>')
def download_file(path, filename):
    return send_from_directory(path, filename, as_attachment=True)

# Load Train Set, Test Set


In [4]:
# #TODO add function to load train set
# #Uploading Training Set
# creating an instance of a flask web application

train_set_path = ''

# @app.route('/trainModelPage', methods=['GET','POST'])
# def upload_Train_set():
#     if request.method == "POST":
#         if 'file' not in request.files:
#             flash('No file parts')
#             #print("sdsd")
#             return redirect(request.url)
#         filesize =   request.cookies.get('filesize')  
#         file =  request.files['file']
#         print(f"Filesize: {filesize}")
#         print(file)
#         # #print("ssss", app.config['UPLOAD_FOLDER'])
#         # train_set = UploadFiles(app.config['UPLOAD_FOLDER'], request.files)
#         # #print("aAS")
#         # train_set_path = train_set.upload()
#         # #print(train_set_path)
#         # os.path.exists(os.path.abspath(train_set_path))
#         # #print('ss', train_set_path)

#         # session['train_set_path'] = train_set_path

#         # images_to_display = []
#         # counter = 0
#         # for dirname, _, filenames in os.walk(train_set_path):
#         #     for filename in filenames:
#         #         if counter == 8:
#         #             break
#         #         temp_list = []
#         #         temp_list.append(dirname)
#         #         temp_list.append(filename)
#         #         # imagePath = os.path.join(dirname, filename)
#         #         images_to_display.append(temp_list)
#         #         counter = counter + 1

#         # #print(images_to_display)

#         print("File uploaded")

#         res = make_response(jsonify({"message": f"{file.filename} uploaded"}), 200)

#         return res
#     return render_template('train_model.html') #,images_to_display=images_to_display
#     #return redirect(url_for('train_and_save'))

# # #TODO add function to load test set
# # #Uploading Testing Set
@app.route('/uploadTestSet', methods=['POST'])
def upload_Test_set():
    if 'file' not in request.files:
        flash('No file parts')
        #print("sdsd")
        return redirect(request.url)
    #print("ssss", app.config['UPLOAD_FOLDER'])
    test_set = UploadFiles(app.config['UPLOAD_FOLDER'], request.files)
    #print("aAS")
    test_set_path = test_set.upload()
    #print(test_set_path)
    os.path.exists(os.path.abspath(test_set_path))
    #print('ss', test_set_path)

    session['test_set_path'] = test_set_path

    images_to_display = []
    counter = 0
    for dirname, _, filenames in os.walk(test_set_path):
        for filename in filenames:
            if counter == 8:
                break
            temp_list = []
            temp_list.append(dirname)
            temp_list.append(filename)
            # imagePath = os.path.join(dirname, filename)
            images_to_display.append(temp_list)
            counter = counter + 1

    #print(images_to_display)
    return render_template('retreive_images_old.html',
                           images_to_display=images_to_display)
    #return redirect(url_for('train_and_save'))

In [5]:
classes_data = pd.read_csv("E:/IRMA X-ray dataset/ImageCLEFmed2009_test_codes.03.csv",sep=';') #usecols=['image_id', '05_class'] 
# Preview the first 5 lines of the loaded data 
# print(len(data))
classes_data = classes_data.set_index("image_id", drop = False)
# print(classes_data)
classes_data.head()

# print(classes_data.at[14642,"05_class"])


# preparing intermediate DataFrame
datasetPath = Path('E:/IRMA X-ray dataset/ImageCLEFmed2009_test.03/ImageCLEFmed2009_test.03')
df = pd.DataFrame()
myimagepath = [] 
classesname = []
filesname = []
directoryname = []
for dirname, _, filenames in os.walk(datasetPath):
    for filename in filenames:
        directoryname.append(dirname)
        imagePath = os.path.join(dirname, filename)
        filesname.append(filename)
        myimagepath.append(imagePath)
        imageID = util.getImageID(filename)
        #print(imageID, classeName)
        classesname.append(classes_data.at[int(imageID),"05_class"])
        #print(imagePath, className)        
df['image'] ,df['classe'], df['filename'],df['directoryname']= [myimagepath,classesname,filesname,directoryname]
df.head()

#getLabels From Csv
#Check if there are some settings to change 

,image,classe,filename,directoryname
0,E:\IRMA X-ray dataset\ImageCLEFmed2009_test.03...,15,10358.png,E:\IRMA X-ray dataset\ImageCLEFmed2009_test.03...
1,E:\IRMA X-ray dataset\ImageCLEFmed2009_test.03...,31,11198.png,E:\IRMA X-ray dataset\ImageCLEFmed2009_test.03...
2,E:\IRMA X-ray dataset\ImageCLEFmed2009_test.03...,56,11416.png,E:\IRMA X-ray dataset\ImageCLEFmed2009_test.03...
3,E:\IRMA X-ray dataset\ImageCLEFmed2009_test.03...,\N,11424.png,E:\IRMA X-ray dataset\ImageCLEFmed2009_test.03...
4,E:\IRMA X-ray dataset\ImageCLEFmed2009_test.03...,9,11426.png,E:\IRMA X-ray dataset\ImageCLEFmed2009_test.03...


In [6]:
# preparing intermediate DataFrame
# Loag data label from folders names
################################################################Labels From folder name####################################################
# datasetPath = Path('E:/Medical MNIST/New folder') 
# df = pd.DataFrame()
# myimagepath = [] 
# classesname = []
# filesname = []
# directoryname = []
# for dirname, _, filenames in os.walk(datasetPath):
#     for filename in filenames:
#         directoryname.append(dirname)
#         imagePath = os.path.join(dirname, filename)
#         filesname.append(filename)
#         myimagepath.append(imagePath)
#         c = util.getImageClass(imagePath)
#         #print(filename, c, )
#         classesname.append(c)
#         # print(filesname)        
# df['image'] ,df['classe'], df['filename'],df['directoryname']= [myimagepath,classesname,filesname,directoryname]
# df.head()

In [7]:
#summary(ConvAutoencoder_v2().to(device),(3,512,512))

# Train Model and Save

In [8]:
@app.route('/train', methods=['POST'])
def train_and_save():
    optimizer = request.form['optimizer']
    learnin_rate = request.form['lr']
    epochs = request.form['epochs']
    batch_size = request.form['batchsize']
    modelName = request.form['modelName']
    print(modelName)

    model_name_info_ = modelName +'_'+  optimizer +'_'+ learnin_rate +'_'+ epochs  +'_'+ batch_size
    
    
    # #TODO add function called train with parameters
    #Check if first if there any information in the session
    if "train_set_path" in session:
        train_set_path = session['train_set_path']
        #print('sldflsdjflk')
    else:
        #print('aaaaa')
        render_template('train_model.html')


        
    #TODO add function to preparing trainset intermediate DataFrame
    # preparing intermediate DataFrame
    datasetPath = Path(train_set_path)
    df = pd.DataFrame()
    myfilename = [] 
    for dirname, _, filenames in os.walk(datasetPath):
        for filename in filenames:
            #print(os.path.join(dirname, filename))
            myfilename.append(os.path.join(dirname, filename))
    df['image'] = myfilename
    df.head()
    #print(df)

    EPOCHS = int(epochs)
    NUM_BATCHES = int(batch_size)
    RETRAIN = False

    # EPOCHS,NUM_BATCHES,df, 
    train_set, validate_set = util.prepare_data(DF=df)

    dataloaders = {'train': DataLoader(train_set, batch_size=NUM_BATCHES, shuffle=True, num_workers=1) ,
                    'val':DataLoader(validate_set, batch_size=NUM_BATCHES, num_workers=1)
                    }

    #To show images
    # images = next(iter(DataLoader(train_set, batch_size=NUM_BATCHES, shuffle=True, num_workers=1)))
    # helper.imshow(images[31], normalize=False)

    dataset_sizes = {'train': len(train_set),'val':len(validate_set)}

    model = ConvAutoencoder_v2().to(device)

    criterion = nn.MSELoss()
    # Observe that all parameters are being optimized
    optimizer = torch.optim.Adam(model.parameters(), lr=float(learnin_rate)) if (optimizer=="Adam") else torch.optim.SGD(model.parameters(), lr=float(learnin_rate)) 
    # Decay LR by a factor of 0.1 every 7 epochs
    exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)#this was commented

    #Training Fucntions
    model, optimizer, loss = util.train_model(
                    model=model,
                    dataloaders = dataloaders,
                    dataset_sizes = dataset_sizes,
                    criterion=criterion, 
                    optimizer=optimizer, 
                    scheduler=exp_lr_scheduler,
                    num_epochs=EPOCHS)

    
    #Save the trained model 
    torch.save({ 'epoch': EPOCHS, 'model_state_dict': model.state_dict(), 'optimizer_state_dict': optimizer.state_dict(), 'loss': loss, }, app.config['UPLOAD_FOLDER']+'/Models/'+model_name_info_+'.pt')

    return redirect(url_for('dashboard_page'))

### Loading Trained Model

In [9]:
# https://stackoverflow.com/questions/36158058/torch-save-tensor-to-csv-file

In [10]:
# # #TODO add function to load query image
# # #Uploading Query Image

#Uploading query image
# @app.route('/uploadQueryImage', methods=['POST'])
# def upload_image():
#     imagefile = request.files['imagefile']
#     image_path = QUERY_IMAGE_UPLOAD_FOLDER + imagefile.filename
#     imagefile = Utils.pre_processing(imagefile)
#     imagefile.save(image_path)
#     #redirect(url_for("extract_query_image_features", image_path=image_path))
#     return redirect(url_for('home_page'))

# #Uploading query image
# @app.route('/uploadQueryImage', methods=['POST'])
# def upload_image():
#     imagefile = request.files['file']
#     number_of_retreived_images = request.form['numberRetreivedImages']
#     model = request.form['model']
#     fileName = imagefile.filename
#     image_path = QUERYIMAGEPATH +  fileName
#     fileName = imagefile.filename
#     #img = Image.open(imagefile)
#     #imagefile = util.pre_processing(img)
#     imagefile.save(image_path)

#     print(QUERYIMAGEPATH,fileName)
#     #redirect(url_for("extract_query_image_features", image_path=image_path))
#     return render_template('retreive_images.html',p=QUERYIMAGEPATH, f= fileName)

def getFileName(query_image_file_name):
    x = query_image_file_name.split(".")
    query_image_file_name = x[0]
    return query_image_file_name

def getQueryImageFolderPath(query_image_file_name,test_set_path):
    query_image_file_name = getFileName(query_image_file_name) 
    for dirname, _, filenames in os.walk(test_set_path):
        for filename in filenames:
            filename = getFileName(filename) 
            if query_image_file_name == filename:
                return str(dirname)

# test = getQueryImageFolderPath('000001.jpeg','E:/Medical MNIST/New folder')

def getQueryImageClassFromFolder(queryImagePath):
    queryImagePath = queryImagePath.split("\\")
    queryImagePath = queryImagePath[len(queryImagePath)-1]
    return queryImagePath 

# getQueryImageClassFromFolder(test)


@app.route('/search', methods=['POST'])
def search():
    imagefile = request.files['file']
    number_of_retreived_images = request.form['numberRetreivedImages']
    selected_model = request.form['model']
    fileName = imagefile.filename
    image_path = QUERYIMAGEPATH +  fileName
    fileName = imagefile.filename
    imagefile.save(image_path)
    #get transformations
    print(selected_model)
    loaded_model = util.load_model_from_dir(selected_model)

    # if bool(index_dict):
	#     print('List is not empty')
    # else:
	#     print('List is empty')

    _, classes, _, index_dict ,filesNames, diractoriesNames= util.features_extraction_and_index_creation(df, transformations,loaded_model)
    query_image_latent_features = util.get_image_latent_features(image_path, transformations, loaded_model)

    results = util.perform_search(query_image_latent_features, index_dict, int(number_of_retreived_images))
    print(results)


    query_image_name = getFileName(fileName)
    query_image_folder = getQueryImageFolderPath(query_image_name,'E:/Medical MNIST/New folder')
    query_image_class = getQueryImageClassFromFolder(str(query_image_folder))


    # print('*********************************** image_path', image_path)
    # print('*********************************** query_image_class', query_image_class)
    images_to_display = []

    # loop over the results
    for (d, j) in results:
        temp_list = []
        temp_list.append(diractoriesNames[j])
        temp_list.append(filesNames[j])
        img_classe = classes[j]
        temp_list.append(img_classe)
        border_coloer = 'rgb(35, 171, 63)' if (query_image_class==img_classe) else 'rgb(226, 44, 44)'
        temp_list.append(border_coloer)
        temp_list.append(d)
        images_to_display.append(temp_list)

    
    
    return render_template('retreive_images_old.html',images_to_display=images_to_display,p=QUERYIMAGEPATH, f= fileName)

In [11]:
app.run()
# #Running the app
# if __name__ == "__main__":
#     app.run(debug=True)
%tb

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://a4f1fba93c05.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
